In [1]:
from time import sleep
import google.generativeai as genai
import configparser
import json
import re
import pandas as pd

/home/gcasado0/proyectos/chatbot/chatbot/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# leer archivo config.ini
config = configparser.ConfigParser()
config.read('config.ini')
GOOGLE_API_KEY = config['GOOGLE']['GOOGLE_API_KEY']
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

In [3]:
consulta = """
"Buenas tardes, soy Extranjero Venezolano y antes no tenía DNI y me otorgaron el CUIL/CUIT anses que tienen actualmente, pero, ya me llego el DNI y actualice mi CUIL/CUIT
Datos actuales:
  DNI: 96372499
  CUIL/CUIT: 27963724999

COMO PODRÁ DARSE CUENTA NECESITO QUE MI CUENTA SE ACTUALICE CON MIS DOCUMENTOS ACTUALIZADOS 
"""

prompt = f"""
Determine cinco temas que se están tratando en el siguiente texto, que está delimitado por tres comillas invertidas.
Haga que cada elemento tenga una o dos palabras.
Formatee su respuesta como una lista de elementos separados por comas.
Indique el sentimiento general del texto.
Indique si el remitente está expresando enojo.
Texto: '''{consulta}'''
Salida en formato JSON:
"Temas": 
"Sentimiento": 
"Enojo": "Si/No"
"""
response = model.generate_content(
        prompt,
        generation_config = genai.GenerationConfig(
        temperature=0.0,
        )
    )

print(response.text)

```json
{
  "Temas": "Actualización de datos, DNI, CUIL/CUIT, Cuenta, Documentos",
  "Sentimiento": "Preocupación, solicitud",
  "Enojo": "No"
}
```



Recorrer la lista de consultas de habilitaciones y obtener tematicas claves y sentimientos de las mismas


In [4]:
data = pd.read_excel('encontraste_lo_que_buscabas-procesado.xlsx')
data.head(5)

,Serial number,Submission ID,Creado,Página actual,Remote IP address,Notes,Email,Mensaje,URL,respuesta,Resumen,Temas,Sentimientos,Enojo,Principal,Secundaria
0,166305,2109849,2024-03-26 00:01:08,NaN,186.13.96.200,NaN,ivanagutierrez92@gmail.com,NaN,https://www.rosario.gob.ar/inicio/gestionesmr?...,si,NaN,NaN,NaN,NaN,NaN,NaN
1,166306,2109857,2024-03-26 00:03:42,NaN,186.152.252.13,NaN,NaN,NaN,https://www.rosario.gob.ar/inicio/obtener-el-c...,si,NaN,NaN,NaN,NaN,NaN,NaN
2,166307,2109858,2024-03-26 00:05:14,NaN,181.117.29.235,NaN,NaN,NaN,https://www.rosario.gob.ar/inicio/agenda/busca...,si,NaN,NaN,NaN,NaN,NaN,NaN
3,166308,2109866,2024-03-26 00:09:54,NaN,181.169.48.96,NaN,monicazrzn@hotmail.com,Ya respondí que ese auto no figura a mi nombre...,https://www.rosario.gob.ar/inicio/infraccion/0...,comentario,Auto vendido hace 12 años,"Auto, venta, titularidad","Frustración, impaciencia",No,Multas,Titularidad
4,166309,2109882,2024-03-26 00:21:33,NaN,186.13.96.162,NaN,julianjoseabrt@gmail.com,NaN,https://www.rosario.gob.ar/inicio/usuariomr?co...,si,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# recorrer todo el dataframe data y extraer el resumen, temas, sentimientos, enojo, complejidad y no sabe actividad

for i, row in data.iterrows():

    # ResourceExhausted: 429 Resource has been exhausted (e.g. check quota)    
    consulta = data['Mensaje'][i]
    # si la consulta es nula o el largo de la consulta es menor a 10 caracteres no la proceso
    if pd.isnull(consulta) or len(consulta) < 10:
        continue 
    print(consulta)   
    prompt = f"""
    Extraer un resumen de lo que quiere el remitente en la siguiente texto, que está delimitada por tres comillas invertidas.
    Extraer los temas principales de la consulta.
    Haga que cada elemento tenga una o dos palabras. Formatee su respuesta como una lista de elementos separados por comas.
    Indique el sentimiento general del texto.
    Indique si el remitente está expresando enojo.    

    Texto: '''{consulta}'''
    Salida en formato JSON:
    "Resumen":
    "Temas": 
    "Sentimientos": 
    "Enojo": "Si/No"
    """
    response = model.generate_content(
        prompt,
        generation_config = genai.GenerationConfig(
        temperature=0.0,
        )
                                      )
    print(response.text)
    # Extraer el contenido JSON del Markdown
    markdown_text = response.text

    # Usar una expresión regular para extraer el contenido JSON
    json_match = re.search(r'```json\n(.*?)\n```', markdown_text, re.DOTALL)
    if json_match:
        json_string = json_match.group(1)
        try:
            # Convertir la cadena de texto a un objeto JSON
            analisis = json.loads(json_string)
            # Agregar respuesta al dataframe data por cada campo de response.text
            data.loc[i, 'Resumen'] = analisis.get('Resumen', 'None')
            data.loc[i, 'Temas'] = analisis.get('Temas', [])            
            data.loc[i, 'Sentimientos'] = analisis.get('Sentimientos', 'None')
            data.loc[i, 'Enojo'] = analisis.get('Enojo', 'None')            

        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON: {e}")
    else:
        print("No se encontró contenido JSON en la respuesta.")

    sleep(5) # para no exceder la cuota de Google de 15 requests por minuto
    if i>300:
        break

# guardar el dataframe data con las respuestas en un archivo Excel
data.to_excel('encontraste_lo_que_buscabas-procesado.xlsx', index=False)


In [ ]:
data = pd.read_excel('encontraste_lo_que_buscabas-procesado.xlsx')
data.head(5)

In [5]:
# recorrer todo el dataframe data y extraer el resumen, temas, sentimientos, enojo, complejidad y no sabe actividad

for i, row in data.iterrows():

    consulta = data['Mensaje'][i]
    url_origen = data['URL'][i]
    # si la consulta es nula o el largo de la consulta es menor a 10 caracteres no la proceso
    if pd.isnull(consulta) or len(consulta) < 10:
        continue 


    # Expresión regular para capturar desde "/inicio/" hasta antes de los parámetros
    pattern = r"/inicio/[^?]+"

    # Buscar el patrón en la URL
    match = re.search(pattern, url_origen)

    # Obtener el resultado
    if match:
        url_result = match.group(0)
    else:
        url_result = url_origen

    print(i, url_result)
    print(consulta)
    prompt = f"""
    Se le proporcionarán consultas de vecinos al sitio web de la municipalidad.
    Clasifique cada consulta en una categoría principal y una categoría secundaria en base al texto y a la URL de origen, que está delimitado por tres comillas invertidas.
    Categorías principales: Reclamos, Multas, Licencia de Conducir, Turnos, Impuestos, Perfil Digital, Salud, Indefinido

    Categorías secundarias de Reclamos:
    Alumbrado
    Arbolado
    Higiene urbana
    Indefinido 

    Categorías secundarias de Multas:
    Pagar
    Turnos
    Titularidad
    Descargos
    Indefinido

    Categorías secundarias de Licencia de conducir:
    Turnos
    Consultas
    Cenat y/o Sellados
    Indefinido

    Categorías secundarias de Impuestos:
    TGI
    Patente
    Indefinido

    Categorías secundarias de Salud:
    Sin turnos
    Error en el sistema
    Reclamo
    Indefinido

    Categorías secundarias de Perfil Digital:
    Actualización de datos
    Indefinido
    
    Categorías secundarias de Salud:
    Vacunas
    Indefinido
    
    Texto: '''{consulta}'''
    URL origen: '''{url_result}'''
    
    Salida en formato JSON:
    "Principal":
    "Secundaria": 
    """
    response = model.generate_content(
        prompt,
        generation_config = genai.GenerationConfig(
        temperature=0.0,
        )
                                      )
    print(response.text)
    # Extraer el contenido JSON del Markdown
    markdown_text = response.text

    # Usar una expresión regular para extraer el contenido JSON
    json_match = re.search(r'```json\n(.*?)\n```', markdown_text, re.DOTALL)
    if json_match:
        json_string = json_match.group(1)
        try:
            # Convertir la cadena de texto a un objeto JSON
            analisis = json.loads(json_string)
            # Agregar respuesta al dataframe data por cada campo de response.text
            data.loc[i, 'Principal'] = analisis.get('Principal', 'None')
            data.loc[i, 'Secundaria'] = analisis.get('Secundaria', [])            

        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON: {e}")
    else:
        print("No se encontró contenido JSON en la respuesta.")
        
    sleep(5) # para no exceder la cuota de Google de 15 requests por minuto
            
    if i>300:
        break

# guardar el dataframe data con las respuestas en un archivo Excel
data.to_excel('encontraste_lo_que_buscabas-procesado.xlsx', index=False)


3 /inicio/infraccion/0be6a140-9f8c-4f05-be0d-6d20cf6b3f95
Ya respondí que ese auto no figura a mi nombre,fue vendido hace más de 12 años y figura en Rosario yo vendí a Roberto Martin
```json
{
  "Principal": "Multas",
  "Secundaria": "Titularidad"
}
```

9 /inicio/avisosmrdetalle/1765948/1
No pude hacer el descargo
```json
{
  "Principal": "Multas",
  "Secundaria": "Descargos"
}
```

10 /inicio/turnosmr
Hola, no puedo sacar un turno para la renovación  del carnet de conducir. Gracias
```json
{
  "Principal": "Licencia de conducir",
  "Secundaria": "Turnos"
}
```

13 /inicio/renovar-medio-boleto-estudiantil
Renovación del medio boleto estudiantil
```json
{
  "Principal": "Indefinido",
  "Secundaria": "Indefinido"
}
```

14 /inicio/renovar-medio-boleto-estudiantil
Renovación de medio boleto estudiantil de Delfina Martínez, D.N.I 55712987, nro de tarjeta sube, 6061 2684 9600 2821
```json
{
  "Principal": "Indefinido",
  "Secundaria": "Indefinido"
}
```

16 /inicio/usuariomr
Buenas tardes,